# DetecTreeRGB Tiling

This notebook **loads**:
* Tiffs of RGB forest imagery
* Shapefiles of manually delineated tree crowns

This notebook **explores**:
* Tiff bounds

This notebook **does**:
* Tiling of large tiffs
* Tiling of large shapefiles
* Conversion of tiled tiff to png
* Conversion of tiled shapefile to GeoJSON
* Adjustment of these so that they align sensibly with origin (0,0) - this is tricky and still not solved properly...

Once tiled...

We can then train a Mask R-CNN model which learns how to delineate trees exactly using manual crowns as training data.

This is given in training.ipynb

Predicting with a trained model is done in predicting.ipynb





In [ ]:
# mount google drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# may need to restart kernel after this cell is 

!pip -q install pygeos

     |████████████████████████████████| 2.1 MB 5.0 MB/s 


In [ ]:
# install necessary geospatial packages

!pip -q install rasterio
!pip -q install fiona
!pip -q install geopandas
!pip -q install pycrs
!pip -q install descartes 
!pip -q install pypng

     |████████████████████████████████| 19.3 MB 271 kB/s 
     |████████████████████████████████| 16.7 MB 5.1 MB/s 
     |████████████████████████████████| 1.0 MB 4.8 MB/s 
     |████████████████████████████████| 6.3 MB 52.0 MB/s 
     |████████████████████████████████| 48 kB 2.3 MB/s 


In [ ]:
# necessary basic libraries
import pandas as pd
import numpy as np
import cv2
import random
import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np
import json
import png
import glob

# geospatial libraries
import rasterio
import geopandas
from geopandas.tools import sjoin
import fiona
from rasterio.plot import show
from rasterio.mask import mask
from shapely.geometry import box
import geopandas as gpd
from fiona.crs import from_epsg
import pycrs
import descartes

# import more geospatial libraries
import rasterio
from rasterio.transform import from_origin
import rasterio.features

import fiona

from shapely.geometry import shape, mapping, box
from shapely.geometry.multipolygon import MultiPolygon

/usr/local/lib/python3.7/dist-packages/geopandas/_compat.py:115: UserWarning: The Shapely GEOS version (3.10.2-CAPI-1.16.0) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


## Exploration of the data

In [ ]:
### let's first read in the data.
### Firstly the tiff file of our area of interest, secondly our shapefile of manually delineated crowns, 
### if we are training the model rather than simply using a pre-trained model.

# Read in a tiff file
data = rasterio.open('/content/drive/Shareddrives/detectreeRGB/benchmark/Ortho2015_benchmark/P4_Ortho_2015.tif')

# Read in shapefile of crowns, if training on your own data!
#crowns = geopandas.read_file('/home/jovyan/lustre_scratch/sepilok_data/sep_danum_crowns_no_overlap/all_manual_crowns_no_overlap.shp')

# have a look at the crowns if we like
#crowns

In [ ]:
# let's investigate the tiff, what is the shape? Bounds? Bands? CRS?
# show a plot of it too

print('shape =', data.shape,',', data.bounds, 'and number of bands =', data.count, ',crs =', data.crs)

# have a look if you want (usually slow)
#show(data)

shape = (3470, 3440) , BoundingBox(left=285791.82599999994, bottom=582793.138, right=286135.82599999994, top=583140.1379999999) and number of bands = 3 ,crs = EPSG:3182


In [ ]:
## find x and y origin of the tiff, need to set the north western corner as the origin 

print(data.bounds)
tiff_x_origin = data.bounds[0]
tiff_y_origin = data.bounds[1]
print('Tiff x origin:', tiff_x_origin)
print('Tiff y origin:', tiff_y_origin)

In [ ]:
# just defining a function we are going to use shortly.

def getFeatures(gdf):
        """Function to parse features from GeoDataFrame in such a manner that rasterio wants them"""
        return [json.loads(gdf.to_json())['features'][0]['geometry']]

In [ ]:
# Read in a tiff file, and the csv of the tiles we expect
data = rasterio.open('/content/drive/Shareddrives/detectreeRGB/benchmark/Ortho2015_benchmark/P7_Ortho_2015.tif')

# Read in the shapefile of manual crowns - again, this is only if you are wanting to train with your own crowns
crowns = geopandas.read_file('/content/drive/Shareddrives/detectreeRGB/benchmark/P7_tiled_140222/crowns/P7_4326_crs.shp')


# set the desired buffer, tile width, heght and resolution of the tiff tiles. Suggested values are given below. 
# The buffer will be affected by the area of tree crowns in your region of forest.
buffer = 20
tile_width = 100
tile_height = 100
resolution = 0.1 # in metres per pixel - @James Ball can you get this from the tiff?
scaling = 1/resolution  # scaling parameter to transform the shapefile coordinates so they match the png

for minx in np.arange(data.bounds[0], data.bounds[2], tile_width):
    # perhaps strictly this should be maxy, as we are now indexing from the top
    for miny in np.arange(data.bounds[3], data.bounds[1], -tile_height):
      print(miny)    
      # define the bounding box of the whole tile, including the buffer
      #bbox = box(minx-buffer, miny-buffer, minx+tile_width+buffer, miny+tile_height+buffer)
      
      # with new tiling
      bbox = box(minx-buffer, miny+buffer, minx+tile_width+buffer, miny-tile_height-buffer)

      # define the bounding box of the tile, excluding the buffer (hence selecting just the central part of the tile)
      bbox_central = box(minx, miny, minx+tile_width, miny-tile_height)
      # turn the bounding boxes into geopandas DataFrames
      geo = gpd.GeoDataFrame({'geometry': bbox}, index=[0], crs=from_epsg(4326))
      geo_central = gpd.GeoDataFrame({'geometry': bbox_central}, index=[0], crs=from_epsg(4326))   #3182
      
      ### here we are cropping the tiff to the bounding box of the tile we want
      coords = getFeatures(geo)
      #print(coords)
      
      # define the tile as a mask of the whole tiff with just the bounding box
      out_img, out_transform = mask(data, shapes=coords, crop=True)
      
      # copy the metadata
      out_meta = data.meta.copy()
      #print(out_meta)
      epsg_code = int(data.crs.data['init'][5:])
      #print(epsg_code)
      
      # update the metadata
      out_meta.update({"driver": "GTiff",
                      "height": out_img.shape[1],
                      "width": out_img.shape[2],
                      "transform": out_transform
                      })
      
      # here we are saving the tile as a new tiff, named by the origin of the tile
      out_tif = '/content/drive/Shareddrives/detectreeRGB/benchmark/P7_tiled_140222/tiffs/tile_'+str(minx)+'_'+str(miny)+'.tif'
      with rasterio.open(out_tif, "w", **out_meta) as dest:
                        dest.write(out_img)
      
      # read in the tile we have just saved
      clipped = rasterio.open('/content/drive/Shareddrives/detectreeRGB/benchmark/P7_tiled_140222/tiffs/tile_'+str(minx)+'_'+str(miny)+'.tif')
      # read it as an array
      arr = clipped.read()
      
      # check the shape of the tile if you wish
      #print(arr.shape)
      
      # each band of the tiled tiff is a colour!
      R = arr[0]
      G = arr[1]
      B = arr[2]
      
      # stack up the bands in an order appropriate for saving with cv2, then rescale to the correct 0-255 range for cv2
      
      rgb = np.dstack((B,G,R)) # BGR for cv2
      rgb_rescaled = rgb # scale to image
      
      # save this as jpg or png...we are going for png...again, named with the origin of the specific tile
      # original
      #cv2.imwrite('/content/drive/Shareddrives/detectreeRGB/benchmark/P7_tiled_140222/pngs/tile_'+str(minx)+'_'+str(miny)+'.png', rgb_rescaled)
      # here as a naughty method
      cv2.imwrite('/content/drive/Shareddrives/detectreeRGB/benchmark/training_P7/train/tile_'+str(minx)+'_'+str(miny)+'.png', rgb_rescaled)
      
      img = cv2.imread('/content/drive/Shareddrives/detectreeRGB/benchmark/training_P7/train/tile_'+str(minx)+'_'+str(miny)+'.png')
      #print('png shape:', img.shape) 

      ### now we have dealt with tiling the tiff, we want to deal with tiling the crowns...
      
      ### IF we have manual crowns we are going to use for training
      ### THEN uncomment the following lines of code
      

      #print('crowns:', crowns)
      #print('geo_central', geo_central)

      ### select the crowns that intersect the non-buffered central section of the tile using the inner join
      overlapping_crowns = sjoin(crowns, geo_central, how="inner")
      #print(overlapping_crowns)

      ### translate to 0,0 to overlay on png
      
      # this should now work as a universal approach...we will see
      if minx == data.bounds[0] and miny == data.bounds[3]:
        print('We are in the top left!')
        moved = overlapping_crowns.translate(-minx, -miny+tile_height+buffer)
      elif minx == data.bounds[0] and miny == np.arange(data.bounds[3], data.bounds[1], -100)[-1]:
        print('We are in the bottom left')
        moved = overlapping_crowns.translate(-minx, -miny+(img.shape[0]/scaling)-buffer)
      # need to fix this line! Just can't get the float to add right
      elif miny == np.arange(data.bounds[3], data.bounds[1], -100)[-1]:
        print('We are on the bottom, but not bottom left')
        moved = overlapping_crowns.translate(-minx+buffer, -miny+(img.shape[0]/scaling)-buffer)
      elif minx == data.bounds[0]:
        print('We are along the left hand side, but not top left!')
        moved = overlapping_crowns.translate(-minx, -miny+tile_height+buffer)
      elif miny == data.bounds[3]:
        print('We are along the top, but not top left!')
        moved = overlapping_crowns.translate(-minx+buffer, -miny+tile_height+buffer)
      else:
        print('We are in the middle!')
        moved = overlapping_crowns.translate(-minx+buffer, -miny+tile_height+buffer)

      
      # original line of code here...non universal
      #moved = overlapping_crowns.translate(-minx, -miny+tile_height+buffer)
      #print(moved) 
      #moved.to_file(driver = 'GeoJSON', 
      #                           filename= '/content/drive/Shareddrives/detectreeRGB/benchmark/training_P7/crowns_moved_non_scaled/tile_'+str(minx)+'_'+str(miny)+'.geojson')
      
      
      ### scale to deal with the resolution

      ### @James Ball - this scaling need to be actually correct!
      ### something like width of tile/pixels in tile
      ### Pan has probably done this.
      
      moved_scaled = moved.scale(scaling, scaling, origin=(0, 0)) 
      #print(moved_scaled)
      ### save as a geojson, a format compatible with detectron2, again named by the origin of the tile
      moved_scaled.to_file(driver = 'GeoJSON', 
                                 filename= '/content/drive/Shareddrives/detectreeRGB/benchmark/training_P7/train/tile_'+str(minx)+'_'+str(miny)+'.geojson')
          